In [16]:
import pandas as pd

In [17]:
data=pd.read_csv("concrete_data.csv")

In [18]:
data.sample(5)

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
634,275.0,0.0,0.0,183.0,0.0,1088.0,808.0,28,24.50
156,286.3,200.9,0.0,144.7,11.2,1004.6,803.7,56,72.99
611,277.0,0.0,0.0,191.0,0.0,968.0,856.0,14,21.26
405,500.0,0.0,0.0,151.0,9.0,1033.0,655.0,28,69.84
831,154.0,144.0,112.0,220.0,10.0,923.0,658.0,28,16.50


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   cement                         1030 non-null   float64
 1   blast_furnace_slag             1030 non-null   float64
 2   fly_ash                        1030 non-null   float64
 3   water                          1030 non-null   float64
 4   superplasticizer               1030 non-null   float64
 5   coarse_aggregate               1030 non-null   float64
 6   fine_aggregate                 1030 non-null   float64
 7   age                            1030 non-null   int64  
 8   concrete_compressive_strength  1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


In [20]:
data.isnull().sum()

cement                           0
blast_furnace_slag               0
fly_ash                          0
water                            0
superplasticizer                 0
coarse_aggregate                 0
fine_aggregate                   0
age                              0
concrete_compressive_strength    0
dtype: int64

In [21]:
X=data.drop(columns=['concrete_compressive_strength'])
Y=data['concrete_compressive_strength']

In [22]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [23]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [24]:
X_train.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age
196,194.7,0.0,100.5,165.6,7.5,1006.4,905.9,28
631,325.0,0.0,0.0,184.0,0.0,1063.0,783.0,7
81,318.8,212.5,0.0,155.7,14.3,852.1,880.4,3
526,359.0,19.0,141.0,154.0,10.9,942.0,801.0,3
830,162.0,190.0,148.0,179.0,19.0,838.0,741.0,28


In [25]:
STD=StandardScaler()
X_train_scaled=STD.fit_transform(X_train)
X_test_scaled=STD.transform(X_test)

In [26]:
ADA=AdaBoostRegressor(
    estimator=DecisionTreeRegressor(
        max_depth=1,
        random_state=42
    ),
    n_estimators=100,
    learning_rate=1.0,
    random_state=42
)

In [27]:
ADA.fit(X_train_scaled,Y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=1, random_state=42),
                  n_estimators=100, random_state=42)

In [28]:
Y_pred=ADA.predict(X_test_scaled)

In [29]:
r2=r2_score(Y_test,Y_pred)
MSE=mean_squared_error(Y_test,Y_pred)
MAE=mean_absolute_error(Y_test,Y_pred)

In [30]:
print("r2 score is",r2)
print("Mean Squared Error is",MSE)
print("Mean Absolute Error is",MAE)

r2 score is 0.5156277641459862
Mean Squared Error is 131.0586623404563
Mean Absolute Error is 9.511210783799546


In [35]:
param_grid={
    'n_estimators':[50,100,200,300],
    'learning_rate':[0.01,0.05,0.1,0.5,1.0],
    'estimator__max_depth':[1,2]
}

In [36]:
grid=GridSearchCV(
    estimator=AdaBoostRegressor(
        estimator=DecisionTreeRegressor(
            max_depth=1,
            random_state=42
        )),
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)
    

In [37]:
grid.fit(X_train_scaled,Y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5,
             estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=1,
                                                                         random_state=42)),
             n_jobs=-1,
             param_grid={'estimator__max_depth': [1, 2],
                         'learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0],
                         'n_estimators': [50, 100, 200, 300]},
             scoring='r2', verbose=2)

In [38]:
best_model=grid.best_estimator_
best_score=grid.best_score_
best_params=grid.best_params_

In [39]:
print("Best score is",best_score)
print("Best parameters are",best_params)

Best score is 0.7244271666023703
Best parameters are {'estimator__max_depth': 2, 'learning_rate': 1.0, 'n_estimators': 200}


In [40]:
Y_pred1=best_model.predict(X_test_scaled)

In [42]:
r21=r2_score(Y_test,Y_pred1)
MAE1=mean_absolute_error(Y_test,Y_pred1)
MSE1=mean_squared_error(Y_test,Y_pred1)

In [43]:
print("r2 score is",r21)
print("Mean Absolute Error is",MAE1)
print("Mean Squared Error is",MSE1)

r2 score is 0.7058316919815756
Mean Absolute Error is 7.460760563691559
Mean Squared Error is 79.59437411575693
